In [ ]:
import tensorflow as tf
import math
import time
from datetime import datetime
 
def conv_op(input,kh,kw,n_out,dh,dw,parameters,name):
    """
    定义卷积层的操作
    :param input: 输入的tensor
    :param kh:卷积核的高
    :param kw:卷积核的宽
    :param n_out:输出通道数（即卷积核的数量）
    :param dh:步长的高
    :param dw:步长的宽
    :param parameters:参数列表
    :param name:层的名字
    :return:返回卷积层的结果
    """
 
    n_in = input.get_shape()[-1].value  #通道数
 
    with tf.name_scope(name) as scope:
        kernel =tf.get_variable(scope+'w',
                                shape=[kh,kw,n_in,n_out],dtype=tf.float32,
                                initializer=tf.contrib.layers.xavier_initializer_conv2d())
        conv=tf.nn.conv2d(input,kernel,[1,dh,dw,1],padding='SAME')
        biases=tf.Variable(tf.constant(0.0,shape=[n_out],dtype=tf.float32),
                           trainable=True,name='b')
        z=tf.nn.bias_add(conv,biases) # wx+b
        activation =tf.nn.relu(z,name=scope)
        parameters +=[kernel,biases]
        return activation
 
def fc_op(input,n_out,parameters,name):
    """
    定义全连接层操作
    注意：卷积层的结果要做扁平化才能和fc层相连接
    :param input: 输入的tensor
    :param n_out: 输出通道数（即神经元的数量）
    :param parameters: 参数列表
    :param name: 层的名字
    :return: 返回全连接层的结果
    """
 
    n_in=input.get_shape()[-1].value
 
    with tf.name_scope(name) as scope:
        kernel =tf.get_variable(scope+'w',
                                shape=[n_in,n_out],dtype=tf.float32,
                                initializer=tf.contrib.layers.xavier_initializer() )
 
        biases = tf.Variable(tf.constant(0.1, shape=[n_out], dtype=tf.float32),
                             trainable=True, name='b')
        activation=tf.nn.relu(tf.matmul(input,kernel)+biases, name=scope)
        parameters +=[kernel,biases]
        return activation
 
 
def maxPool_op(input,kh,kw,dh,dw,name):
    return tf.nn.max_pool(input,ksize=[1,kh,kw,1],strides=[1,dh,dw,1],
                          padding='SAME',name=name)
 
 
 
def vggNet(input,keep_prob):
    parameters =[]
 
    #conv1段
    conv1_1 =conv_op(input,kh=3,kw=3,n_out=64,dh=1,dw=1,
                    parameters=parameters,name='conv1_1')
    conv1_2 =conv_op(conv1_1, kh=3, kw=3, n_out=64, dh=1, dw=1,
                      parameters=parameters, name='conv1_2')
    pool1 =maxPool_op(conv1_2,kh=2,kw=2,dh=2,dw=2,name='pool1')
 
    # conv2段
    conv2_1 = conv_op(pool1, kh=3, kw=3, n_out=128, dh=1, dw=1,
                      parameters=parameters, name='conv2_1')
    conv2_2 = conv_op(conv2_1, kh=3, kw=3, n_out=128, dh=1, dw=1,
                      parameters=parameters, name='conv2_2')
    pool2 = maxPool_op(conv2_2, kh=2, kw=2, dh=2, dw=2, name='pool2')
 
    # conv3段
    conv3_1 = conv_op(pool2, kh=3, kw=3, n_out=256, dh=1, dw=1,
                      parameters=parameters, name='conv3_1')
    conv3_2 = conv_op(conv3_1, kh=3, kw=3, n_out=256, dh=1, dw=1,
                      parameters=parameters, name='conv3_2')
    conv3_3 = conv_op(conv3_2, kh=3, kw=3, n_out=256, dh=1, dw=1,
                      parameters=parameters, name='conv3_3')
    pool3 = maxPool_op(conv3_3, kh=2, kw=2, dh=2, dw=2, name='pool3')
 
 
    # conv4段
    conv4_1 = conv_op(pool3, kh=3, kw=3, n_out=512, dh=1, dw=1,
                      parameters=parameters, name='conv4_1')
    conv4_2 = conv_op(conv4_1, kh=3, kw=3, n_out=512, dh=1, dw=1,
                      parameters=parameters, name='conv4_2')
    conv4_3 = conv_op(conv4_2, kh=3, kw=3, n_out=512, dh=1, dw=1,
                      parameters=parameters, name='conv4_3')
    pool4 = maxPool_op(conv4_3, kh=2, kw=2, dh=2, dw=2, name='pool4')
 
    # conv5段
    conv5_1 = conv_op(pool4, kh=3, kw=3, n_out=512, dh=1, dw=1,
                      parameters=parameters, name='conv5_1')
    conv5_2 = conv_op(conv5_1, kh=3, kw=3, n_out=512, dh=1, dw=1,
                      parameters=parameters, name='conv5_2')
    conv5_3 = conv_op(conv5_2, kh=3, kw=3, n_out=512, dh=1, dw=1,
                      parameters=parameters, name='conv5_3')
    pool5 = maxPool_op(conv5_3, kh=2, kw=2, dh=2, dw=2, name='pool5')
 
    #将最后一个卷积层的结果扁平化:每个样本占一行
    conv_shape=pool5.get_shape()
    col=conv_shape[1].value *conv_shape[2].value * conv_shape[3].value
    flat=tf.reshape(pool5, [-1,col],name='flat')
 
    # fc6段
    fc6 =fc_op(input=flat,n_out=4096,parameters=parameters,name='fc6')
    fc6_dropout =tf.nn.dropout(fc6,keep_prob,name='fc6_drop')
 
    # fc7段
    fc7 = fc_op(input=fc6_dropout, n_out=4096, parameters=parameters, name='fc7')
    fc7_dropout = tf.nn.dropout(fc7, keep_prob, name='fc7_drop')
 
    # fc8段：最后一个全连接层，使用softmax进行处理得到分类输出概率
    fc8=fc_op(input=fc7_dropout,n_out=1000,parameters=parameters,name='fc8')
    softmax =tf.nn.softmax(fc8)
    predictions =tf.arg_max(softmax,1)
    return predictions,softmax,fc8,parameters
 
 
def time_compute(session, target, feed,info_string):
    num_batch = 100 #100
    num_step_burn_in = 10  # 预热轮数，头几轮迭代有显存加载、cache命中等问题可以因此跳过
    total_duration = 0.0  # 总时间
    total_duration_squared = 0.0
    for i in range(num_batch + num_step_burn_in):
        start_time = time.time()
        _ = session.run(target,feed_dict=feed )
        duration = time.time() - start_time
        if i >= num_step_burn_in:
            if i % 10 == 0:  # 每迭代10次显示一次duration
                print("%s: step %d,duration=%.5f " % (datetime.now(), i - num_step_burn_in, duration))
            total_duration += duration
            total_duration_squared += duration * duration
    time_mean = total_duration / num_batch
    time_variance = total_duration_squared / num_batch - time_mean * time_mean
    time_stddev = math.sqrt(time_variance)
    # 迭代完成，输出
    print("%s: %s across %d steps,%.3f +/- %.3f sec per batch " %
          (datetime.now(), info_string, num_batch, time_mean, time_stddev))
 
 
def main():
    with tf.Graph().as_default():
        """仅使用随机图片数据 测试前馈和反馈计算的耗时"""
        image_size = 224
        batch_size = 2   #32
 
        images = tf.Variable(tf.random_normal([batch_size, image_size, image_size, 3],
                                              dtype=tf.float32, stddev=0.1))
 
        keep_prob=tf.placeholder(tf.float32)
        predictions,softmax,fc8, parameters = vggNet(images,keep_prob)
 
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
 
        """
        AlexNet forward 计算的测评
        传入的target:fc8（即最后一层的输出）
        优化目标：loss
        使用tf.gradients求相对于loss的所有模型参数的梯度
 
 
        AlexNet Backward 计算的测评
        target:grad
 
        """
        time_compute(sess, target=fc8, feed={keep_prob:1.0},info_string="Forward")
 
        obj = tf.nn.l2_loss(fc8)
        grad = tf.gradients(obj, parameters)
        time_compute(sess, grad, feed={keep_prob:0.5},info_string="Forward-backward")
 
 
if __name__ == "__main__":
    main()